# Annotation tool for time series data

By: Stefania Russo, Kris Villez
Copyright: 2018, distributed with BSD3 license 

## The challenge

In the context of the ADASen project, we want to address research questions regarding the utility of supervised and unsupervised machine learning models in anomaly detection for environmental systems. We have therefore selected a range of anomaly detection methods for benchmarking on data sets produced by six infrastructures at Eawag.

Critical to the benchmarking is the availability of fully labelled training and test data sets of normal and abnormal behavior in environmental data. 
An annotation tool has therefore being developed to perform the labelling procedure.

This notebook shows an application of the labelling procedure to time series data. Here, each time series is a univariate 24h signal

Each series is visualised as a 24h time series.

## Current method

Below are described the steps for data access, data preparation, visualization and labelling procedure.

- The data is in the form of .csv data files. Each data file consists of many 24h sets across 2 sensors.

    - if missing values are already replaced with NaNs
    - If none, replace missing values with NaNs
    - Decide if removing dates with Missing Values
    - Perform Annotation

- The labelling procedure starts and the first plots are displayed. The plots at the top are univariate sensor signals, where the bottom plot shows a collection of these signals.  

- The annotation tool allows the labelling expert to interactivelly select multiple portions of the time series by moving through the data with the mouse cursor.

- Each time the button 'Next' is clicked, all the selected areas (time index and sensor value) are saved together with information about the date stamp date. At the end of the procedure, the user can easily access to the anomaly labels in an easy manner.

- When the process is over, the plots need to be closed and then the cell 'Save labelled data' hs to be run 

- Note: if the user wants to change any of his selections, he needs to move forward to the next plot by clicking 'Next', perform a selection of the anomalous data, and then go back and restart.


# Usage (general)
- Create folder called "labels" into your data folder
- Select data file name_of_file
- Select Case (1 or 2): 
    - CASE 1 will display 3 plots: the plots at the top are univariate sensor signals, where the bottom plot shows the difference between these signals. 
    - CASE 2 will display 3 plots: the plots at the top are univariate sensor signals, where the bottom plot shows the signals combined. 
- Run the cells
- After performing the annotation, close the plot and run the last cell 'Save labelled data'
- Note: if the user wants to change any of his selections, he needs to move forward to the next plot by clicking 'Next', perform a selection of the anomalous data, and then go back and restart.



# Usage as .ipyn script
- Run each cell in the notebook
- Select data file name_of_file + enter
- Select Case + enter
- After performing the annotation, close the plot.
- Restart

# Iniziatization

In [1]:
# Import Statements
%run functions/startup.py
random.seed(4)

All libraries have been loaded.


In [2]:
notebook_type = 'ipynb'    #ipynb   #py

In [3]:
case_from_terminal = input('Please select Case (1: GAK (p3,p4)  2: Pressure T1 T2 (p1,p2)):  ')
text_from_terminal = input("Please enter the file name:  ")  # Python 3

### Options

In [4]:
Case = int(case_from_terminal)   

In [5]:
path_base = str(Path.cwd())
folder = '/data_examples/NEST_data/'  # Select data folder

path_all = path_base + folder
save_path = path_all     # Destination folder to for labelled data
name_of_file = text_from_terminal

In [6]:
name_of_file1 = name_of_file + ".csv"
name_of_file_l1 =  'labels/' + name_of_file + '_labels_'
name_of_file_l1_time = 'labels/' + name_of_file + '_labels_time'

completePath = os.path.join(path_all, name_of_file1) 

In [7]:
print ('Now working with : ', folder, ' file: ', name_of_file)

Now working with :  /data_examples/NEST_data/  file:  s1


In [8]:
# ADD zeros and ones with dictionary mapping

mapper_dict = {'left_only': 0, 'both': 1}

def mp(entry):
    """
    maps new values
    """
    return mapper_dict[entry] if entry in mapper_dict else entry
mp = np.vectorize(mp)

# Load data and basic sanity checks

In [9]:
# Load data
df = pd.read_csv(completePath)
df.head()
df2 = df.copy(deep=True)

sr0 = df2.keys()[2]
sr1 = df2.keys()[3]
print('Sensor names:',sr0,',', sr1)

Sensor names: p3 , p4


In [10]:
# df2['Datetime'] = df2['day'] + ' ' + df2['hour']
# df2['Datetime_'] = [x for x in (pd.to_datetime([i for i in df2['Datetime']], format='%d.%m.%Y %H:%M:%S'))] 
# df3 = df2.resample('10S', on='Datetime_', base=10).mean()

# print(df2)

df2['Datetime_'] = pd.to_datetime(df2['day'] + ' ' + df2['hour'], format='%d.%m.%Y %H:%M:%S')
df2.set_index('Datetime_', inplace=True)

df2.index += pd.Timedelta(seconds=10)

# This will round down each timestamp to the nearest 10-second mark
df2.index = df2.index - pd.to_timedelta(df2.index.second % 10, unit='s')

print(df2.dtypes)

df3 = df2[['p3', 'p4']].resample('10S').mean()

day      object
hour     object
p3      float64
p4      float64
dtype: object


In [11]:
df3 = df3.reset_index()
df3['day'] = [x.date() for x in df3['Datetime_']] 
df3['time'] = [x.time() for x in df3['Datetime_']] 

In [74]:
df4 = df3.copy(deep=True)
df4.set_index(['day','time'], inplace=True)

df_bf_00 = df4[sr0]
df_bf_01 = df4[sr1]
df_bf_02 = df4[sr0] - df4[sr1]

print(df_bf_00)
print("-"*50)
print(df_bf_01)
print("-"*50)
print(df_bf_02)

df4.drop(columns ='Datetime_', inplace=True)
df2 = df4.copy(deep=True)

day         time    
2018-06-01  00:00:10    91.6
            00:00:20    91.6
            00:00:30    91.6
            00:00:40    91.6
            00:00:50    91.6
                        ... 
2018-06-05  23:59:20    91.6
            23:59:30    91.6
            23:59:40    91.6
            23:59:50    91.6
2018-06-06  00:00:00    91.6
Name: p3, Length: 43200, dtype: float64
--------------------------------------------------
day         time    
2018-06-01  00:00:10    12.6
            00:00:20    12.6
            00:00:30    12.6
            00:00:40    12.6
            00:00:50    12.6
                        ... 
2018-06-05  23:59:20    12.5
            23:59:30    12.5
            23:59:40    12.5
            23:59:50    12.5
2018-06-06  00:00:00    12.5
Name: p4, Length: 43200, dtype: float64
--------------------------------------------------
day         time    
2018-06-01  00:00:10    79.0
            00:00:20    79.0
            00:00:30    79.0
            00:00:40    79.0
 

## Basic sanity checks

In [134]:
# Accessing dates
i_date = df2.index.get_level_values(0)                                      # get all dates
idx_date = np.unique(df2.index.get_level_values(0), return_index=True)[1]      # get index of unique dates
date_list = i_date[idx_date]   # get list of all dates

# Dates and times
data_df2 = df2.copy()

data_time = []
for pl_i in idx_date:                             # create data_time indeces to have access later
    time = data_df2.loc[i_date[pl_i]].index
    data_time.append(time)                        # associated to every date segment
    
time_int = [np.linspace(1, 8640, num = 8640, dtype=int) for _ in range(len(date_list)-1)]

# Fixed error here. The number of date_list is not equal to time_int
print(len(date_list))
print(len(time_int))

# Revised approach to ensure time_int matches the length of actual data for each date
# time_int = [np.linspace(1, len(data_df2.loc[date]), num=len(data_df2.loc[date]), dtype=int) for date in date_list]

lengths = [len(arr) for arr in time_int]
print(lengths)


min_length = min(len(time_int), len(date_list))
time_int, date_list = time_int[:min_length], date_list[:min_length]

lengths = [len(arr) for arr in time_int]
print(lengths)

print(f"Number of unique dates in date_list: {len(date_list)}")

print(df)

min_val = min(min(df[df.keys()[2]]),min(df[df.keys()[3]]))
max_val = max(max(df[df.keys()[2]]),max(df[df.keys()[3]]))

print(min_val)
print(max_val)

# date_list=date_list[:len(date_list)-1]
print(time_int)
print(date_list)

6
5
[8640, 8640, 8640, 8640, 8640]
[8640, 8640, 8640, 8640, 8640]
Number of unique dates in date_list: 5
              day      hour    p3    p4
0      01.06.2018  00:00:00  91.6  12.6
1      01.06.2018  00:00:10  91.6  12.6
2      01.06.2018  00:00:20  91.6  12.6
3      01.06.2018  00:00:30  91.6  12.6
4      01.06.2018  00:00:40  91.6  12.6
...           ...       ...   ...   ...
43195  05.06.2018  23:59:10  91.6  12.5
43196  05.06.2018  23:59:20  91.6  12.5
43197  05.06.2018  23:59:30  91.6  12.5
43198  05.06.2018  23:59:40  91.6  12.5
43199  05.06.2018  23:59:50  91.6  12.5

[43200 rows x 4 columns]
12.3
93.7
[array([   1,    2,    3, ..., 8638, 8639, 8640]), array([   1,    2,    3, ..., 8638, 8639, 8640]), array([   1,    2,    3, ..., 8638, 8639, 8640]), array([   1,    2,    3, ..., 8638, 8639, 8640]), array([   1,    2,    3, ..., 8638, 8639, 8640])]
Index([2018-06-01, 2018-06-02, 2018-06-03, 2018-06-04, 2018-06-05], dtype='object', name='day')


# Plotting

In [88]:
#get_ipython().run_line_magic('matplotlib', 'tk')

if notebook_type == 'ipynb':
    %matplotlib tk

if notebook_type == 'py':
    import matplotlib as mpl
    mpl.use('Qt5Agg')

data1 = []
data2 = []
data123 = []

itera = date_list

gs = gridspec.GridSpec(2, 2)

fig = plt.figure()
#plt.axis([0, 24, -3, 100])


ax1 = fig.add_subplot(gs[0, 0]) # row 0, col 0
ax2 = fig.add_subplot(gs[0, 1]) # row 0, col 1
ax4 = fig.add_subplot(gs[1, :]) # row 1, span all columns

ax1.set_title(sr0, fontdict=None, pad=None)
ax2.set_title(sr1, fontdict=None, pad=None)
full = sr0 + ' '+ sr1
ax4.set_title(full, fontdict=None, pad=None)

fig.suptitle(str(date_list[0]), fontsize=12)

print(f"Y-axis limits: min_val = {min_val}, max_val = {max_val}")

ax1.set_ylim([min_val,max_val])
ax2.set_ylim([min_val,max_val])
ax4.set_ylim([min_val,max_val])

# print(len(ax1))
# print(len(ax2))

# for pl_i in range(len(date_list)): 

#     # Get the x and y values to be plotted
#     x_values = time_int[pl_i]
#     y_values_00 = df_bf_00[date_list[pl_i]].values
#     y_values_01 = df_bf_01[date_list[pl_i]].values

#     # Print the values being plotted
#     print("-"*50)
#     print(f"Plotting for date: {date_list[pl_i]}")
#     print(f"x_values (length {len(x_values)}): {x_values}")
#     print(f"y_values_00 (length {len(y_values_00)}): {y_values_00}")
#     print(f"y_values_01 (length {len(y_values_01)}): {y_values_01}")
#     print("-"*50)

#     min_length = min(len(x_values), len(y_values_00), len(y_values_01))
#     x_values = x_values[:min_length]
#     y_values_00 = y_values_00[:min_length+1]
#     y_values_01 = y_values_01[:min_length+1]

#     ax1.plot(x_values, y_values_00, '#C0C0C0', lw=2)
#     ax2.plot(x_values, y_values_01, '#C0C0C0', lw=2)
    

for pl_i in range(len(date_list)):
    date = date_list[pl_i]
    
    # Get the x and y values to be plotted
    x_values = time_int[pl_i]
    y_values_00 = df_bf_00.loc[date].values
    y_values_01 = df_bf_01.loc[date].values

    # Print the values being plotted
    print(f"Plotting for date: {date}")
    print(f"x_values (length {len(x_values)}): {x_values}")
    print(f"y_values_00 (length {len(y_values_00)}): {y_values_00}")
    print(f"y_values_01 (length {len(y_values_01)}): {y_values_01}")

    # Ensure x and y values are aligned by length
    min_length = min(len(x_values), len(y_values_00), len(y_values_01))
    x_values = x_values[:min_length]
    y_values_00 = y_values_00[:min_length]
    y_values_01 = y_values_01[:min_length]

    # Plot the values
    ax1.plot(x_values, y_values_00, '#C0C0C0', lw=2)
    ax2.plot(x_values, y_values_01, '#C0C0C0', lw=2)


# l, = ax1.plot(time_int[0], df_bf_00[date_list[0]].values, '#1E90FF', lw=2)     #the first one is the one in blue
# l2, = ax2.plot(time_int[0], df_bf_01[date_list[0]].values, '#8B008B')

# Plot initial data points for the first date in blue and purple
x_values_initial = time_int[0]
y_values_initial_00 = df_bf_00.loc[date_list[0]].values
y_values_initial_01 = df_bf_01.loc[date_list[0]].values

# Ensure initial x and y values are aligned by length
min_length_initial = min(len(x_values_initial), len(y_values_initial_00), len(y_values_initial_01))
x_values_initial = x_values_initial[:min_length_initial]
y_values_initial_00 = y_values_initial_00[:min_length_initial]
y_values_initial_01 = y_values_initial_01[:min_length_initial]

# Plot the initial values
l, = ax1.plot(x_values_initial, y_values_initial_00, '#1E90FF', lw=2)
l2, = ax2.plot(x_values_initial, y_values_initial_01, '#8B008B')

Y-axis limits: min_val = 12.3, max_val = 93.7
Plotting for date: 2018-06-01
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8639): [91.6 91.6 91.6 ... 92.7 92.4 92.2]
y_values_01 (length 8639): [12.6 12.6 12.6 ... 17.5 16.8 16.1]
Plotting for date: 2018-06-02
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.9 91.7 91.8 ... 91.6 91.6 91.6]
y_values_01 (length 8640): [15.5 15.1 14.7 ... 12.5 12.5 12.6]
Plotting for date: 2018-06-03
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.6 91.6 91.6 ... 91.6 91.6 91.6]
y_values_01 (length 8640): [12.6 12.5 12.6 ... 12.6 12.6 12.5]
Plotting for date: 2018-06-04
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.6 91.6 91.6 ... 91.7 91.6 91.6]
y_values_01 (length 8640): [12.5 12.6 12.5 ... 12.6 12.5 12.6]
Plotting for date: 2018-06-05
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]


In [109]:
#get_ipython().run_line_magic('matplotlib', 'tk')

if notebook_type == 'ipynb':
    %matplotlib tk

if notebook_type == 'py':
    import matplotlib as mpl
    mpl.use('Qt5Agg')

data1 = []
data2 = []
data123 = []

itera = date_list

gs = gridspec.GridSpec(2, 2)

fig = plt.figure()
#plt.axis([0, 24, -3, 100])


ax1 = fig.add_subplot(gs[0, 0]) # row 0, col 0
ax2 = fig.add_subplot(gs[0, 1]) # row 0, col 1
ax4 = fig.add_subplot(gs[1, :]) # row 1, span all columns

ax1.set_title(sr0, fontdict=None, pad=None)
ax2.set_title(sr1, fontdict=None, pad=None)
full = sr0 + ' '+ sr1
ax4.set_title(full, fontdict=None, pad=None)

fig.suptitle(str(date_list[0]), fontsize=12)

print(f"Y-axis limits: min_val = {min_val}, max_val = {max_val}")

ax1.set_ylim([min_val,max_val])
ax2.set_ylim([min_val,max_val])
ax4.set_ylim([min_val,max_val])

# print(len(ax1))
# print(len(ax2))

# for pl_i in range(len(date_list)): 

#     # Get the x and y values to be plotted
#     x_values = time_int[pl_i]
#     y_values_00 = df_bf_00[date_list[pl_i]].values
#     y_values_01 = df_bf_01[date_list[pl_i]].values

#     # Print the values being plotted
#     print("-"*50)
#     print(f"Plotting for date: {date_list[pl_i]}")
#     print(f"x_values (length {len(x_values)}): {x_values}")
#     print(f"y_values_00 (length {len(y_values_00)}): {y_values_00}")
#     print(f"y_values_01 (length {len(y_values_01)}): {y_values_01}")
#     print("-"*50)

#     min_length = min(len(x_values), len(y_values_00), len(y_values_01))
#     x_values = x_values[:min_length]
#     y_values_00 = y_values_00[:min_length+1]
#     y_values_01 = y_values_01[:min_length+1]

#     ax1.plot(x_values, y_values_00, '#C0C0C0', lw=2)
#     ax2.plot(x_values, y_values_01, '#C0C0C0', lw=2)
    

for pl_i in range(len(date_list)):
    date = date_list[pl_i]
    
    # Get the x and y values to be plotted
    x_values = time_int[pl_i]
    y_values_00 = df_bf_00.loc[date].values
    y_values_01 = df_bf_01.loc[date].values

    # Print the values being plotted
    print(f"Plotting for date: {date}")
    print(f"x_values (length {len(x_values)}): {x_values}")
    print(f"y_values_00 (length {len(y_values_00)}): {y_values_00}")
    print(f"y_values_01 (length {len(y_values_01)}): {y_values_01}")

    # Ensure x and y values are aligned by length
    min_length = min(len(x_values), len(y_values_00), len(y_values_01))
    x_values = x_values[:min_length]
    y_values_00 = y_values_00[:min_length]
    y_values_01 = y_values_01[:min_length]

    # Plot the values
    ax1.plot(x_values, y_values_00, '#C0C0C0', lw=2)
    ax2.plot(x_values, y_values_01, '#C0C0C0', lw=2)


# l, = ax1.plot(time_int[0], df_bf_00[date_list[0]].values, '#1E90FF', lw=2)     #the first one is the one in blue
# l2, = ax2.plot(time_int[0], df_bf_01[date_list[0]].values, '#8B008B')

# Plot initial data points for the first date in blue and purple
x_values_initial = time_int[0]
y_values_initial_00 = df_bf_00.loc[date_list[0]].values
y_values_initial_01 = df_bf_01.loc[date_list[0]].values

# Ensure initial x and y values are aligned by length
min_length_initial = min(len(x_values_initial), len(y_values_initial_00), len(y_values_initial_01))
x_values_initial = x_values_initial[:min_length_initial]
y_values_initial_00 = y_values_initial_00[:min_length_initial]
y_values_initial_01 = y_values_initial_01[:min_length_initial]

# Plot the initial values
l, = ax1.plot(x_values_initial, y_values_initial_00, '#1E90FF', lw=2)
l2, = ax2.plot(x_values_initial, y_values_initial_01, '#8B008B')


###########################
if Case == 1:
    ll1, = ax4.plot(x_values_initial, y_values_initial_00.values, '#C0C0C0')
    ll2, = ax4.plot(x_values_initial, y_values_initial_01.values, '#C0C0C0')
    ll3, = ax4.plot(time_int[0], df_bf_02[date_list[0]].values, '#31a354')   # add difference4 plot

if Case == 2:
    ll1, = ax4.plot(x_values_initial, y_values_initial_00.values, '#1E90FF')
    ll2, = ax4.plot(x_values_initial, y_values_initial_01, '#8B008B')

############### Buttons widget  ####################

class Index(object):
    ind = 0

    def next(self, event):
        self.ind += 1
        i = self.ind % len(itera)

        #ydata0 will be the plot alone
        ydata1 = df_bf_00[date_list[i]].values   
        ydata2 = df_bf_01[date_list[i]].values 
        ydata3 = df_bf_02[date_list[i]].values 
        
        xdata = time_int[i]          
        
        l.set_ydata(ydata1)
        l.set_xdata(xdata)
        l2.set_ydata(ydata2)
        l2.set_xdata(xdata)
        
        if Case == 1:
            ll1.set_ydata(ydata1)
            ll2.set_ydata(ydata2)
            ll3.set_ydata(ydata3)
            ll1.set_xdata(xdata) 
            ll2.set_xdata(xdata)
            ll3.set_xdata(xdata)
        
        if Case == 2:                
            ll1.set_ydata(ydata1)
            ll2.set_ydata(ydata2)
            ll1.set_xdata(xdata) 
            ll2.set_xdata(xdata)
        
        
        if (i == (0)):
            fig.suptitle('End of data files - restarting with data file ' + str(date_list[i]), fontsize=12)
        else: 
            fig.suptitle(str(date_list[i]), fontsize=12)
            
        plt.draw()

    def prev(self, event):
        self.ind -= 1
        i = self.ind % len(itera)
        
        #ydata0 will be the plot alone
        ydata1 = df_bf_00[date_list[i]].values 
        ydata2 = df_bf_01[date_list[i]].values 
        ydata3 = df_bf_02[date_list[i]].values 
        
        xdata = time_int[i]  
        
        l.set_ydata(ydata1)
        l.set_xdata(xdata)
        
        l2.set_ydata(ydata2)
        l2.set_xdata(xdata)
        
        if Case == 1:
            ll1.set_ydata(ydata1)
            ll2.set_ydata(ydata2)
            ll3.set_ydata(ydata3)
            ll1.set_xdata(xdata) 
            ll2.set_xdata(xdata)
            ll3.set_xdata(xdata)
        
        if Case == 2:                
            ll1.set_ydata(ydata1)
            ll2.set_ydata(ydata2)
            ll1.set_xdata(xdata) 
            ll2.set_xdata(xdata)
        

        if (i == (0)):
            fig.suptitle('End of data files - restarting with data file ' + str(date_list[i]), fontsize=12)
        else: 
            fig.suptitle(str(date_list[i].date()), fontsize=12)
            
        plt.draw()

callback = Index()

axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.81, 0.05, 0.1, 0.075])
bnext = Button(axnext, 'Next')
bnext.on_clicked(callback.next)

bprev = Button(axprev, 'Previous')
bprev.on_clicked(callback.prev)

"""
valore = '11'
def presskey(event):
    print('Pressed key = ', event.key)
    #sys.stdout.flush()    
    global valore 
    valore = event.key       
    return valore
"""

def onselect1(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y = df_bf_00[date_list[callback.ind % len(itera)]].values 
    today = date_list[callback.ind % len(itera)]
   
    indmin1, indmax1 = np.searchsorted(x, (xmin, xmax))
    indmax1 = min(len(x) - 1, indmax1)
    thisx = x[indmin1:indmax1]
    thisy = y[indmin1:indmax1]    
    nplist = np.array([today.date() for i in range(len(thisx))])
        
    a1 = np.c_[nplist, thisx, thisy]
    global data1
    data1.extend(a1)
    #np.savetxt(completeName_label_1, data1)

    ax1.axvspan(xmin, xmax, facecolor='red', alpha=0.5)
    plt.draw()


        

def onselect2(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y = df_bf_01[date_list[callback.ind % len(itera)]].values 
    today = date_list[callback.ind % len(itera)]
    
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    thisx = x[indmin:indmax]
    thisy = y[indmin:indmax]
    nplist = np.array([today.date() for i in range(len(thisx))])
    
    a2 = np.c_[nplist, thisx, thisy]
    global data2
    data2.extend(a2)

    ax2.axvspan(xmin, xmax, facecolor='red', alpha=0.5)
    plt.draw()
    

def onselect4(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y1 = df_bf_00[date_list[callback.ind % len(itera)]].values 
    y2 = df_bf_01[date_list[callback.ind % len(itera)]].values
    today = date_list[callback.ind % len(itera)]
    
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    
    thisx = x[indmin:indmax]
    thisy1 = y1[indmin:indmax]
    thisy2 = y2[indmin:indmax]
    nplist = np.array([today.date() for i in range(len(thisx))])
        
    # save
    a123 = np.c_[nplist, thisx, thisy1, thisy2]
    global data123
    data123.extend(a123)

    ax4.axvspan(xmin, xmax, facecolor='red', alpha=0.5)
    plt.draw()


    
"""
# Connect key event to figure
fig.canvas.mpl_connect('key_press_event',presskey)
"""

#class1 = Onselect_1()

spans1 = SpanSelector(ax1, onselect1, 'horizontal', useblit=False,
                      props=dict(alpha=0.5, facecolor='red'))
span2 = SpanSelector(ax2, onselect2, 'horizontal', useblit=True,
                    props=dict(alpha=0.5, facecolor='red'))
span4 = SpanSelector(ax4, onselect4, 'horizontal', useblit=True,
                    props=dict(alpha=0.5, facecolor='red'))


########################################
if notebook_type == 'py':
    # added
    plt.show()
########################################


Y-axis limits: min_val = 12.3, max_val = 93.7
Plotting for date: 2018-06-01
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8639): [91.6 91.6 91.6 ... 92.7 92.4 92.2]
y_values_01 (length 8639): [12.6 12.6 12.6 ... 17.5 16.8 16.1]
Plotting for date: 2018-06-02
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.9 91.7 91.8 ... 91.6 91.6 91.6]
y_values_01 (length 8640): [15.5 15.1 14.7 ... 12.5 12.5 12.6]
Plotting for date: 2018-06-03
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.6 91.6 91.6 ... 91.6 91.6 91.6]
y_values_01 (length 8640): [12.6 12.5 12.6 ... 12.6 12.6 12.5]
Plotting for date: 2018-06-04
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.6 91.6 91.6 ... 91.7 91.6 91.6]
y_values_01 (length 8640): [12.5 12.6 12.5 ... 12.6 12.5 12.6]
Plotting for date: 2018-06-05
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [128]:
# Matplotlib backend initialization
if notebook_type == 'ipynb':
    %matplotlib tk
elif notebook_type == 'py':
    import matplotlib as mpl
    mpl.use('Qt5Agg')

# Initialize data storage lists
data1 = []
data2 = []
data123 = []

# Iterator for date list
itera = date_list

# Create GridSpec layout for subplots
gs = gridspec.GridSpec(2, 2)
fig = plt.figure()

# Add subplots
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax4 = fig.add_subplot(gs[1, :])

# Set titles
ax1.set_title(sr0, fontdict=None, pad=None)
ax2.set_title(sr1, fontdict=None, pad=None)
full = sr0 + ' ' + sr1
ax4.set_title(full, fontdict=None, pad=None)

# Set super title
fig.suptitle(str(date_list[0]), fontsize=12)

# Set y-axis limits and print them
print(f"Y-axis limits: min_val = {min_val}, max_val = {max_val}")
ax1.set_ylim([min_val, max_val])
ax2.set_ylim([min_val, max_val])
ax4.set_ylim([min_val, max_val])

# Plot data for each date
for pl_i in range(len(date_list)):
    date = date_list[pl_i]
    x_values = time_int[pl_i]
    y_values_00 = df_bf_00.loc[date].values
    y_values_01 = df_bf_01.loc[date].values

    print(f"Plotting for date: {date}")
    print(f"x_values (length {len(x_values)}): {x_values}")
    print(f"y_values_00 (length {len(y_values_00)}): {y_values_00}")
    print(f"y_values_01 (length {len(y_values_01)}): {y_values_01}")

    min_length = min(len(x_values), len(y_values_00), len(y_values_01))
    x_values = x_values[:min_length]
    y_values_00 = y_values_00[:min_length]
    y_values_01 = y_values_01[:min_length]

    ax1.plot(x_values, y_values_00, '#C0C0C0', lw=2)
    ax2.plot(x_values, y_values_01, '#C0C0C0', lw=2)

# Plot initial data points for the first date
x_values_initial = time_int[0]
y_values_initial_00 = df_bf_00.loc[date_list[0]].values
y_values_initial_01 = df_bf_01.loc[date_list[0]].values
y_values_initial_02 = df_bf_02.loc[date_list[0]].values

min_length_initial = min(len(x_values_initial), len(y_values_initial_00), len(y_values_initial_01))
x_values_initial = x_values_initial[:min_length_initial]
y_values_initial_00 = y_values_initial_00[:min_length_initial]
y_values_initial_01 = y_values_initial_01[:min_length_initial]
y_values_initial_02 = y_values_initial_02[:min_length_initial]


l, = ax1.plot(x_values_initial, y_values_initial_00, '#1E90FF', lw=2)
l2, = ax2.plot(x_values_initial, y_values_initial_01, '#8B008B')

# Plot additional data based on Case
if Case == 1:
    ll1, = ax4.plot(x_values_initial, y_values_initial_00, '#C0C0C0')
    ll2, = ax4.plot(x_values_initial, y_values_initial_01, '#C0C0C0')
    ll3, = ax4.plot(x_values_initial, y_values_initial_02, '#31a354')

if Case == 2:
    ll1, = ax4.plot(x_values_initial, y_values_initial_00, '#1E90FF')
    ll2, = ax4.plot(x_values_initial, y_values_initial_01, '#8B008B')

# Button widget for navigation
class Index(object):
    ind = 0

    def next(self, event):
        self.ind += 1
        i = self.ind % len(itera)
        ydata1 = df_bf_00.loc[date_list[i]].values
        ydata2 = df_bf_01.loc[date_list[i]].values 
        ydata3 = df_bf_02.loc[date_list[i]].values 
        xdata = time_int[i]
        
        l.set_ydata(ydata1)
        l.set_xdata(xdata)
        l2.set_ydata(ydata2)
        l2.set_xdata(xdata)
        
        if Case == 1:
            ll1.set_ydata(ydata1)
            ll2.set_ydata(ydata2)
            ll3.set_ydata(ydata3)
            ll1.set_xdata(xdata)
            ll2.set_xdata(xdata)
            ll3.set_xdata(xdata)
        
        if Case == 2:
            ll1.set_ydata(ydata1)
            ll2.set_ydata(ydata2)
            ll1.set_xdata(xdata)
            ll2.set_xdata(xdata)
        
        if i == 0:
            fig.suptitle('End of data files - restarting with data file ' + str(date_list[i]), fontsize=12)
        else:
            fig.suptitle(str(date_list[i]), fontsize=12)
            
        plt.draw()

    def prev(self, event):
        self.ind -= 1
        i = self.ind % len(itera)
        ydata1 = df_bf_00.loc[date_list[i]].values
        ydata2 = df_bf_01.loc[date_list[i]].values 
        ydata3 = df_bf_02.loc[date_list[i]].values 
        xdata = time_int[i]
        
        l.set_ydata(ydata1)
        l.set_xdata(xdata)
        
        l2.set_ydata(ydata2)
        l2.set_xdata(xdata)
        
        if Case == 1:
            ll1.set_ydata(ydata1)
            ll2.set_ydata(ydata2)
            ll3.set_ydata(ydata3)
            ll1.set_xdata(xdata)
            ll2.set_xdata(xdata)
            ll3.set_xdata(xdata)
        
        if Case == 2:
            ll1.set_ydata(ydata1)
            ll2.set_ydata(ydata2)
            ll1.set_xdata(xdata)
            ll2.set_xdata(xdata)
        
        if i == 0:
            fig.suptitle('End of data files - restarting with data file ' + str(date_list[i]), fontsize=12)
        else:
            fig.suptitle(str(date_list[i]), fontsize=12)
            
        plt.draw()

callback = Index()

axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.81, 0.05, 0.1, 0.075])
bnext = Button(axnext, 'Next')
bnext.on_clicked(callback.next)

bprev = Button(axprev, 'Previous')
bprev.on_clicked(callback.prev)

# Span selectors for interactive range selection
def onselect1(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y = df_bf_00.loc[date_list[callback.ind % len(itera)]].values 
    today = date_list[callback.ind % len(itera)]
    
    indmin1, indmax1 = np.searchsorted(x, (xmin, xmax))
    indmax1 = min(len(x) - 1, indmax1)
    thisx = x[indmin1:indmax1]
    thisy = y[indmin1:indmax1]
    nplist = np.array([today.date() for i in range(len(thisx))])
        
    a1 = np.c_[nplist, thisx, thisy]
    global data1
    data1.extend(a1)
    ax1.axvspan(xmin, xmax, facecolor='red', alpha=0.5)
    plt.draw()

def onselect2(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y = df_bf_01.loc[date_list[callback.ind % len(itera)]].values 
    today = date_list[callback.ind % len(itera)]
    
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    thisx = x[indmin:indmax]
    thisy = y[indmin:indmax]
    nplist = np.array([today.date() for i in range(len(thisx))])
    
    a2 = np.c_[nplist, thisx, thisy]
    global data2
    data2.extend(a2)
    ax2.axvspan(xmin, xmax, facecolor='red', alpha=0.5)
    plt.draw()

def onselect4(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y1 = df_bf_00.loc[date_list[callback.ind % len(itera)]].values 
    y2 = df_bf_01.loc[date_list[callback.ind % len(itera)]].values
    today = date_list[callback.ind % len(itera)]
    
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    
    thisx = x[indmin:indmax]
    thisy1 = y1[indmin:indmax]
    thisy2 = y2[indmin:indmax]
    nplist = np.array([today.date() for i in range(len(thisx))])
        
    a123 = np.c_[nplist, thisx, thisy1, thisy2]
    global data123
    data123.extend(a123)
    ax4.axvspan(xmin, xmax, facecolor='red', alpha=0.5)
    plt.draw()

spans1 = SpanSelector(ax1, onselect1, 'horizontal', useblit=False, props=dict(alpha=0.5, facecolor='red'))
span2 = SpanSelector(ax2, onselect2, 'horizontal', useblit=True, props=dict(alpha=0.5, facecolor='red'))
span4 = SpanSelector(ax4, onselect4, 'horizontal', useblit=True, props=dict(alpha=0.5, facecolor='red'))

# Show plot if running in standalone script
if notebook_type == 'py':
    plt.show()


Y-axis limits: min_val = 12.3, max_val = 93.7
Plotting for date: 2018-06-01
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8639): [91.6 91.6 91.6 ... 92.7 92.4 92.2]
y_values_01 (length 8639): [12.6 12.6 12.6 ... 17.5 16.8 16.1]
Plotting for date: 2018-06-02
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.9 91.7 91.8 ... 91.6 91.6 91.6]
y_values_01 (length 8640): [15.5 15.1 14.7 ... 12.5 12.5 12.6]
Plotting for date: 2018-06-03
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.6 91.6 91.6 ... 91.6 91.6 91.6]
y_values_01 (length 8640): [12.6 12.5 12.6 ... 12.6 12.6 12.5]
Plotting for date: 2018-06-04
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]
y_values_00 (length 8640): [91.6 91.6 91.6 ... 91.7 91.6 91.6]
y_values_01 (length 8640): [12.5 12.6 12.5 ... 12.6 12.5 12.6]
Plotting for date: 2018-06-05
x_values (length 8640): [   1    2    3 ... 8638 8639 8640]


In [111]:
plt.close()

## Save labels 

In [129]:
print(len(time_int))
data1 = pd.DataFrame(data1, columns=['day','time_m', sr0])
data2 = pd.DataFrame(data2, columns=['day','time_m', sr1])
data123 = pd.DataFrame(data123, columns=['day','time_m', sr0, sr1])

# # Raw labels
# data1.to_csv(os.path.join(save_path,name_of_file_l1+sr0 + ".csv") )
# data2.to_csv(os.path.join(save_path,name_of_file_l1+sr1 + ".csv") )
# data123.to_csv(os.path.join(save_path,name_of_file_l1+sr0+sr1+ ".csv") )

5


In [130]:
print(len(time_int))
def rem_dup(datal):
    if (len(datal))==0:
        return datal.copy(deep=True)
    #list_org = [str(i) for i in datal['day'].values]
    list_org = [i for i in datal['day'].values]
    states = [list_org[-1]]
    index_keep =[True]

    for i in range(len(list_org)-2,-1,-1):
        if list_org[i]!=list_org[i+1] and list_org[i] not in states:
            states.extend([list_org[i]])
            index_keep.append(True) 

        elif list_org[i]!=list_org[i+1] and list_org[i] in states:
            index_keep.append(False)

        elif list_org[i]==list_org[i+1] and index_keep[len(list_org)-2-i]==False:   # check
            index_keep.append(False)

        elif list_org[i]==list_org[i+1] and list_org[i] in states:
            index_keep.append(True)
            
    index_keep.reverse()
    #index_keep = rem_dup(datal)
    datall = datal[index_keep]
    return datall

5


In [159]:
pd.set_option('mode.chained_assignment', None)

data1_l = rem_dup(data1)
data2_l = rem_dup(data2)
data123_l = rem_dup(data123)

print(data1_l)

Empty DataFrame
Columns: [day, time_m, p3]
Index: []


In [160]:
data_df = data_df2.reset_index()

data_df_s1 = data_df.drop([sr1], axis=1)
data_df_s2 = data_df.drop([sr0], axis=1)
data_df_s3 = data_df.copy()

data_df_s1['time_m'] = np.reshape(time_int, 8640*(len(time_int)))
data_df_s2['time_m'] = np.reshape(time_int, 8640*(len(time_int)))
data_df_s3['time_m'] = np.reshape(time_int, 8640*(len(time_int)))

# Ensure day columns are datetime objects before extracting the date
data_df_s1['day'] = pd.to_datetime(data_df_s1['day'])
data_df_s2['day'] = pd.to_datetime(data_df_s2['day'])
data_df_s3['day'] = pd.to_datetime(data_df_s3['day'])

# Extract the date part
data_df_s1['day'] = data_df_s1['day'].dt.date
data_df_s2['day'] = data_df_s2['day'].dt.date
data_df_s3['day'] = data_df_s3['day'].dt.date

# data_df_s1['day'] = [x.date() for x in data_df_s1['day']] 
# data_df_s2['day'] = [x.date() for x in data_df_s2['day']] 
# data_df_s3['day'] = [x.date() for x in data_df_s3['day']] 

data1_l.drop(columns=[sr0], inplace=True)
data2_l.drop(columns=[sr1], inplace=True)
data123_l.drop(columns=[sr0, sr1], inplace=True)

# data1_l.drop(columns = sr0, inplace=True)
# data2_l.drop(columns = sr1, inplace=True)
# data123_l.drop(columns =[sr0, sr1], inplace=True)

labels_df_1 = pd.merge(data_df_s1, data1_l, on = ['day', 'time_m'], how='left', indicator=True)
labels_df_2 = pd.merge(data_df_s2, data2_l, on = ['day', 'time_m'], how='left', indicator=True)
labels_df_123 = pd.merge(data_df_s3, data123_l, on = ['day', 'time_m'], how='left', indicator=True)

print(labels_df_1.head())
print(labels_df_2.head())
print(labels_df_123.head())

          day      time    p3  time_m     _merge
0  2018-06-01  00:00:10  91.6       1  left_only
1  2018-06-01  00:00:20  91.6       2  left_only
2  2018-06-01  00:00:30  91.6       3  left_only
3  2018-06-01  00:00:40  91.6       4  left_only
4  2018-06-01  00:00:50  91.6       5  left_only
          day      time    p4  time_m     _merge
0  2018-06-01  00:00:10  12.6       1  left_only
1  2018-06-01  00:00:20  12.6       2  left_only
2  2018-06-01  00:00:30  12.6       3  left_only
3  2018-06-01  00:00:40  12.6       4  left_only
4  2018-06-01  00:00:50  12.6       5  left_only
          day      time    p3    p4  time_m     _merge
0  2018-06-01  00:00:10  91.6  12.6       1  left_only
1  2018-06-01  00:00:20  91.6  12.6       2  left_only
2  2018-06-01  00:00:30  91.6  12.6       3  left_only
3  2018-06-01  00:00:40  91.6  12.6       4  left_only
4  2018-06-01  00:00:50  91.6  12.6       5  left_only


In [161]:
labels_df_1 ['_merge'] = mp(labels_df_1['_merge'])
labels_df_2 ['_merge'] = mp(labels_df_2['_merge'])
labels_df_123 ['_merge'] = mp(labels_df_123['_merge'])
labels_df_1 = labels_df_1.rename(index=str, columns={"_merge": "Anomaly"})
labels_df_2 = labels_df_2.rename(index=str, columns={"_merge": "Anomaly"})
labels_df_123 = labels_df_123.rename(index=str, columns={"_merge": "Anomaly"})
labels_df_1.drop(['time_m'], axis=1, inplace=True)
labels_df_2.drop(['time_m'], axis=1, inplace=True)
labels_df_123.drop(['time_m'], axis=1, inplace=True)

In [166]:
if not os.path.exists(save_path):
    os.makedirs(save_path)

print(name_of_file_l1_time)

labels_df_1.to_csv(os.path.join(save_path, name_of_file_l1_time+sr0 + ".csv") )
labels_df_2.to_csv(os.path.join(save_path, name_of_file_l1_time+sr1 + ".csv") )
labels_df_123.to_csv(os.path.join(save_path, name_of_file_l1_time+sr0+sr1 + ".csv") )  

labels/s1_labels_time
